<a href="https://colab.research.google.com/github/Code-Monish/Question-paper-formatter/blob/main/ML_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ASE -B - Hackathon data.csv to ASE -B - Hackathon data.csv


In [ ]:
import pandas as pd
df = pd.read_csv('ASE -B - Hackathon data.csv')
df.head()

,Part ID,Week,Region,Actual Sales
0,1000A30201,202001,Asia,160
1,1000A30201,202002,Asia,110
2,1000A30201,202003,Asia,600
3,1000A30201,202012,Asia,360
4,1000A30201,202013,Asia,474


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228981 entries, 0 to 228980
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Part ID        228981 non-null  object
 1   Week           228981 non-null  int64 
 2   Region         228981 non-null  object
 3   Actual Sales   228981 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 7.0+ MB


In [ ]:
df.describe()

,Week,Actual Sales
count,228981.000000,2.289810e+05
mean,202176.783061,1.069875e+03
std,107.906445,4.210475e+03
min,202001.000000,-2.260000e+04
25%,202103.000000,7.200000e+01
50%,202202.000000,3.420000e+02
75%,202249.000000,1.060000e+03
max,202341.000000,1.280000e+06



# DATA PREPROCESSING

In [ ]:
sum(df.isnull().sum())

0

In [ ]:
print(df.columns)

Index(['Part ID ', 'Week ', 'Region', 'Actual Sales '], dtype='object')


In [ ]:
import numpy as np

df.columns = df.columns.str.strip()  # Remove Strip whitespace from column names

df['Week'] = df['Week'].astype(int)
df['Week'] = pd.to_datetime(df['Week'].astype(str) + '1', format='%Y%W%w', errors='coerce')

# Drop NaT values in Week column
df = df.dropna(subset=['Week'])

df = df.sort_values(by=['Part ID', 'Region', 'Week']).reset_index(drop=True)

In [ ]:
# Time-based features
df['week_number'] = df['Week'].dt.isocalendar().week
df['month'] = df['Week'].dt.month
df['quarter'] = (df['month'] - 1) // 3 + 1
df['year'] = df['Week'].dt.year
df['is_year_end'] = (df['week_number'] == 52).astype(int)

# Cyclical Encoding
df['sin_week'] = np.sin(2 * np.pi * df['week_number'] / 52)
df['cos_week'] = np.cos(2 * np.pi * df['week_number'] / 52)

In [ ]:
# Lag features
def create_lag_features(group):
    group['lag_1'] = group['Actual Sales'].shift(1)
    group['lag_4'] = group['Actual Sales'].shift(4)
    group['rolling_mean_4'] = group['Actual Sales'].shift(1).rolling(window=4).mean()
    group['rolling_std_4'] = group['Actual Sales'].shift(1).rolling(window=4).std()
    group['sales_diff_1'] = group['Actual Sales'].diff(1)
    group['sales_pct_change_1'] = group['Actual Sales'].pct_change(1)
    group['sales_zscore'] = (group['Actual Sales'] - group['Actual Sales'].mean()) / group['Actual Sales'].std()
    group['sales_scaled'] = group['Actual Sales'] / group['Actual Sales'].max()
    return group

In [ ]:
df = df.groupby(['Part ID', 'Region']).apply(create_lag_features).reset_index(drop=True)

<ipython-input-11-8c4384c70f80>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['Part ID', 'Region']).apply(create_lag_features).reset_index(drop=True)


In [ ]:
df.head()

,Part ID,Week,Region,Actual Sales,week_number,month,quarter,year,is_year_end,sin_week,cos_week,lag_1,lag_4,rolling_mean_4,rolling_std_4,sales_diff_1,sales_pct_change_1,sales_zscore,sales_scaled
0,1000A30201,2020-01-06,Asia,160,2,1,1,2020,0,0.239316,0.970942,NaN,NaN,NaN,NaN,NaN,NaN,-1.112779,0.095238
1,1000A30201,2020-01-13,Asia,110,3,1,1,2020,0,0.354605,0.935016,160.0,NaN,NaN,NaN,-50.0,-0.312500,-1.250292,0.065476
2,1000A30201,2020-01-20,Asia,600,4,1,1,2020,0,0.464723,0.885456,110.0,NaN,NaN,NaN,490.0,4.454545,0.097335,0.357143
3,1000A30201,2020-03-23,Asia,360,13,3,1,2020,0,1.0,-0.0,600.0,NaN,NaN,NaN,-240.0,-0.400000,-0.562727,0.214286
4,1000A30201,2020-03-30,Asia,474,14,3,1,2020,0,0.992709,-0.120537,360.0,160.0,307.5,222.916277,114.0,0.316667,-0.249197,0.282143


In [ ]:
df.tail()

,Part ID,Week,Region,Actual Sales,week_number,month,quarter,year,is_year_end,sin_week,cos_week,lag_1,lag_4,rolling_mean_4,rolling_std_4,sales_diff_1,sales_pct_change_1,sales_zscore,sales_scaled
228976,99C21191,2023-10-09,Asia,5184,41,10,4,2023,0,-0.970942,0.239316,10368.0,12049.0,9921.25,3258.114524,-5184.0,-0.500000,-0.819652,0.300000
228977,9C20292,2022-04-25,Asia,1100,17,4,2,2022,0,0.885456,-0.464723,NaN,NaN,NaN,NaN,NaN,NaN,-0.560153,0.009910
228978,9C20292,2022-08-01,Asia,3000,31,8,3,2022,0,-0.568065,-0.822984,1100.0,NaN,NaN,NaN,1900.0,1.727273,-0.524588,0.027027
228979,9C20292,2023-01-02,Asia,9000,1,1,1,2023,0,0.120537,0.992709,3000.0,NaN,NaN,NaN,6000.0,2.000000,-0.412277,0.081081
228980,9C20292,2023-01-09,Asia,111000,2,1,1,2023,0,0.239316,0.970942,9000.0,NaN,NaN,NaN,102000.0,11.333333,1.497018,1.000000


In [ ]:
# Categorical Encoding
from sklearn.preprocessing import LabelEncoder

df['Part_ID_enc'] = LabelEncoder().fit_transform(df['Part ID'])
df['Region_enc'] = LabelEncoder().fit_transform(df['Region'])

In [ ]:
df.columns

Index(['Part ID', 'Week', 'Region', 'Actual Sales', 'week_number', 'month',
       'quarter', 'year', 'is_year_end', 'sin_week', 'cos_week', 'lag_1',
       'lag_4', 'rolling_mean_4', 'rolling_std_4', 'sales_diff_1',
       'sales_pct_change_1', 'sales_zscore', 'sales_scaled', 'Part_ID_enc',
       'Region_enc'],
      dtype='object')

In [ ]:
features = ['Part ID', 'Week', 'Region', 'Actual Sales', 'week_number', 'month',
       'quarter', 'year', 'is_year_end', 'sin_week', 'cos_week', 'lag_1',
       'lag_4', 'rolling_mean_4', 'rolling_std_4', 'sales_diff_1',
       'sales_pct_change_1', 'sales_zscore', 'sales_scaled', 'Part_ID_enc',
       'Region_enc']

Target_feature='Actual Sales'

In [ ]:
df.describe()

/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:52: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,Week,Actual Sales,week_number,month,quarter,year,is_year_end,sin_week,cos_week,lag_1,lag_4,rolling_mean_4,rolling_std_4,sales_diff_1,sales_pct_change_1,sales_zscore,sales_scaled,Part_ID_enc,Region_enc
count,228981,2.289810e+05,228981.0,228981.000000,228981.000000,228981.000000,228981.000000,228981.0,228981.0,222189.000000,207232.000000,207232.000000,207232.000000,222189.000000,2.212850e+05,2.272850e+05,228974.000000,228981.000000,228981.000000
mean,2021-12-31 04:15:50.290198528,1.069875e+03,26.4211,6.422795,2.475293,2021.507881,0.013407,-0.019481,-0.054153,1065.977596,1094.377813,1099.721305,424.561876,4.983888,NaN,9.456813e-18,0.345040,3676.970391,0.287268
min,2020-01-06 00:00:00,-2.260000e+04,1.0,1.000000,1.000000,2020.000000,0.000000,-1.0,-1.0,-22600.000000,-22600.000000,-4724.750000,0.000000,-400000.000000,-inf,-8.307684e+00,-40.400000,0.000000,0.000000
25%,2021-01-18 00:00:00,7.200000e+01,14.0,4.000000,2.000000,2021.000000,0.000000,-0.748511,-0.748511,81.000000,100.000000,140.000000,63.264491,-140.000000,-4.047619e-01,-6.899749e-01,0.133333,1024.000000,0.000000
50%,2022-01-10 00:00:00,3.420000e+02,27.0,6.000000,2.000000,2022.000000,0.000000,-0.0,-0.120537,360.000000,390.000000,441.000000,184.752086,0.000000,0.000000e+00,-1.541709e-01,0.303171,4286.000000,0.000000
75%,2022-12-05 00:00:00,1.060000e+03,38.0,9.000000,3.000000,2022.000000,0.000000,0.663123,0.663123,1080.000000,1152.000000,1152.000000,452.529281,148.000000,5.824176e-01,5.773503e-01,0.500000,5941.000000,1.000000
max,2023-10-09 00:00:00,1.280000e+06,53.0,12.000000,4.000000,2023.000000,1.000000,1.0,1.0,520000.000000,520000.000000,297000.000000,214165.045389,320000.000000,inf,8.792736e+00,1.000000,6791.000000,1.000000
std,NaN,4.210475e+03,14.516527,3.314807,1.082829,1.090569,0.115011,0.709377,0.702479,3011.010655,2965.502896,2734.549246,1330.102310,1908.698570,NaN,9.879980e-01,0.288844,2436.908170,0.452489


In [ ]:
train_list = []
test_list = []

In [ ]:
forecast_duration = 12

In [ ]:
for (part, region), group in df.groupby(['Part ID', 'Region']):
    group = group.sort_values(by='Week').reset_index(drop=True)

    if len(group) > forecast_duration:
        train_data = group.iloc[:-forecast_duration]
        test_data = group.iloc[-forecast_duration:]

        train_list.append(train_data)
        test_list.append(test_data)

In [ ]:
train_df = pd.concat(train_list).reset_index(drop=True)
test_df = pd.concat(test_list).reset_index(drop=True)

In [ ]:
print("Train set shape:", train_df.shape)

Train set shape: (178767, 21)


In [ ]:
print("Test set shape:", test_df.shape)

Test set shape: (35148, 21)


In [ ]:
train_df[train_df['Part ID'] == train_df['Part ID'].iloc[0]].tail(3)

,Part ID,Week,Region,Actual Sales,week_number,month,quarter,year,is_year_end,sin_week,...,lag_1,lag_4,rolling_mean_4,rolling_std_4,sales_diff_1,sales_pct_change_1,sales_zscore,sales_scaled,Part_ID_enc,Region_enc
100,1000A30201,2022-09-19,Asia,240,38,9,3,2022,0,-0.992709,...,240.0,240.0,420.0,360.0,0.0,0.0,-0.892758,0.142857,0,0
101,1000A30201,2022-10-10,Asia,240,41,10,4,2022,0,-0.970942,...,240.0,960.0,420.0,360.0,0.0,0.0,-0.892758,0.142857,0,0
102,1000A30201,2022-10-17,Asia,240,42,10,4,2022,0,-0.935016,...,240.0,240.0,240.0,0.0,0.0,0.0,-0.892758,0.142857,0,0


# **XGBoost Model training **

In [ ]:
!pip install xgboost optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.5 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Training set
X_train = train_df[features]
y_train = train_df[Target_feature]

# Test set
X_test = test_df[features]
y_test = test_df[Target_feature]

In [ ]:
X_train_full, X_val, y_train_full, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
print("NaNs in X_train:", X_train.isnull().sum().sum())
print("NaNs in y_train:", y_train.isnull().sum())

NaNs in X_train: 42831
NaNs in y_train: 0


In [ ]:
# Separate column types
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'category', 'bool']).columns

In [ ]:
X_train[numeric_cols] = X_train[numeric_cols].fillna(X_train[numeric_cols].median())
X_test[numeric_cols] = X_test[numeric_cols].fillna(X_train[numeric_cols].median())

In [ ]:
for col in categorical_cols:
    mode_val = X_train[col].mode()[0]
    X_train[col] = X_train[col].fillna(mode_val)
    X_test[col] = X_test[col].fillna(mode_val)

In [ ]:
print("Remaining NaNs in X_train:", X_train.isnull().sum().sum())
print("Remaining NaNs in X_test:", X_test.isnull().sum().sum())

Remaining NaNs in X_train: 0
Remaining NaNs in X_test: 0


In [ ]:
print(X_train.dtypes)

Part ID                       object
Week                  datetime64[ns]
Region                        object
Actual Sales                   int64
week_number                   UInt32
month                          int32
quarter                        int32
year                           int32
is_year_end                    int64
sin_week                     Float64
cos_week                     Float64
lag_1                        float64
lag_4                        float64
rolling_mean_4               float64
rolling_std_4                float64
sales_diff_1                 float64
sales_pct_change_1           float64
sales_zscore                 float64
sales_scaled                 float64
Part_ID_enc                    int64
Region_enc                     int64
dtype: object


In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("Same number of rows?", X_train.shape[0] == y_train.shape[0])

X_train shape: (178767, 21)
y_train shape: (178767,)
Same number of rows? True


In [ ]:
X_train = X_train.drop(columns=['Part ID', 'Region', 'Week'], errors='ignore')
X_test = X_test.drop(columns=['Part ID', 'Region', 'Week'], errors='ignore')

In [ ]:
import numpy as np

# Replace infinite values with NaN
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

# Impute NaN values using a suitable strategy (e.g., median)
for col in X_train.columns:
    if X_train[col].isnull().any():
        X_train[col] = X_train[col].fillna(X_train[col].median())
        X_test[col] = X_test[col].fillna(X_train[col].median()) # Use train median for test as well

# Now create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [ ]:
n_rounds = 100

In [ ]:
watchlist = [(dtrain, 'train'), (dtest, 'test')]

In [ ]:
# XGBoost regression parameters
params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42
}

In [ ]:
# Train the model
model = xgb.train(params=params, dtrain=dtrain, num_boost_round=n_rounds, evals=watchlist, early_stopping_rounds=10, verbose_eval=True)

[0]	train-rmse:2299.89683	test-rmse:4438.63500
[1]	train-rmse:2075.05324	test-rmse:4318.29273
[2]	train-rmse:1873.37306	test-rmse:4216.66597
[3]	train-rmse:1698.17145	test-rmse:4128.85572
[4]	train-rmse:1545.54934	test-rmse:4061.40696
[5]	train-rmse:1406.50374	test-rmse:4004.95865
[6]	train-rmse:1271.07402	test-rmse:3946.75139
[7]	train-rmse:1149.06092	test-rmse:3897.90479
[8]	train-rmse:1041.93288	test-rmse:3856.98957
[9]	train-rmse:946.65331	test-rmse:3818.67231
[10]	train-rmse:857.90095	test-rmse:3786.45565
[11]	train-rmse:783.85686	test-rmse:3761.95290
[12]	train-rmse:710.62167	test-rmse:3738.65357
[13]	train-rmse:648.65646	test-rmse:3718.44063
[14]	train-rmse:589.60863	test-rmse:3701.54018
[15]	train-rmse:536.25060	test-rmse:3685.71356
[16]	train-rmse:491.03542	test-rmse:3673.64077
[17]	train-rmse:447.79024	test-rmse:3660.16296
[18]	train-rmse:408.77716	test-rmse:3647.85410
[19]	train-rmse:376.75483	test-rmse:3635.97241
[20]	train-rmse:347.64516	test-rmse:3626.77206
[21]	train-rms

In [ ]:
y_pred = model.predict(dtest)

In [ ]:
# Final Metrix
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'RMSE: {rmse:.2f}')
print(f'MAE: {mae:.2f}')
print(f'R² Score: {r2:.4f}')

RMSE: 3535.84
MAE: 66.19
R² Score: 0.3979


In [ ]:
tolerance = 0.10
accuracy = np.mean(np.abs(y_pred - y_test) / y_test < tolerance)
print(f'Model Accuracy : {accuracy * 100:.2f}%')

Model Accuracy (within ±10%): 65.26%


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

# Create a GridSearchCV object
grid_search = GridSearchCV(
    estimator=xgb.XGBRegressor(objective='reg:squarederror', seed=42),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Use a suitable scoring metric
    cv=5,  # Number of cross-validation folds
    verbose=2,  # Show progress
)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)
# ... (Calculate and print evaluation metrics)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, subsample=0.8; total time=   4.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, subsample=0.8; total time=   1.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, subsample=0.8; total time=   1.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, subsample=0.8; total time=   1.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, subsample=0.8; total time=   1.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, subsample=0.9; total time=   1.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, subsample=0.9; total time=   1.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, subsample=0.9; total time=   1.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, subsample=0.9; total time=   3.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_d